# Redes de Hopfield

Una red de Hopfield es una red neuronal que tiene N entradas, N unidades y por lo tanto, N salidas.

Cada salida $O_j$ puede ser calculada como:

$$O_j = signo(\sum_{i=1}^{N} w_{i,j} \xi_i) $$ 

Donde:

$\xi_i$ es la entrada i de la red.  
$w_{i,j}$ es el peso que conecta la entrada i con la unidad j.

La función signo vale 1 cuando su argumento es un número positivo o cero, y -1 cuando su argumento es negativo.

Una red de hopfield puede ser representada de la siguiente manera, según los elementos que venimos utilizando:

<img src="images/hopfield1.png" alt="drawing" />

Para obtener la salida $O_j$ tendremos que multiplicar a cada entrada $i$ por el peso $w_{i,j}$.  
Por claridad solo se anotaron los pesos necesarios para calcular la salida $O_1$.  La red consta de $N^2$ pesos.

## Interpretación del valor de los pesos $w_{i,j}$

Los pesos $w_{i,j}$ en una red entrenada (es decir, con los patrones de referencia almacenados) tienen una medida de cuán correlacionados están los bits $i$ y $j$ en cada patrón de referencia. 

Un $w_{i,j}$ muy positivo indicará que para la mayoría de los patrones de referencia, si $\zeta_i$ vale +1 o -1, entonces $\zeta_j$ también valdrá +1 o -1 respectivamente.  
Un $w_{i,j}$ muy negativo indicará que para la mayoría de los patrones de referencia, si $\zeta_i$ valga +1 o -1, entonces $\zeta_j$ valdrá -1 o +1 (invertirá su signo con respecto a $\zeta_i$).  
Un valor de $w_{i,j}$ cercano a cero indicará que el valor de $\zeta_i$ dará poca información con respecto al valor de $\zeta_j$.  
La salida $O_j$ se obtiene de hacer una "votación" ponderada por estos valores de correlación. Si el resultado de esa votación es positivo (o cero), la salida j será 1. Si el resultado de esa votación es negativo, la salida j será -1.

## Almacenamiento de un patrón de referencia

En un principio, vamos a suponer que solo vamos a almacenar un patrón de referencia $\zeta$.
Como queremos que cada peso $w_{i,j}$ contenga una medida de la correlación entre el bit $i$ y el bit $j$ de $\zeta$, definiremos:

$$ w_{i,j}=\zeta_i.\zeta_j $$

Con esta definición se cumple que cuando $\zeta_i$ sea igual a $\zeta_j$, $w_{i,j}$ valdrá 1. Cuando sean distintos, valdrá -1.

Ejemplo:

$\zeta= [1,1, 1, -1, -1 ,-1]$

Si calculamos la matriz de pesos directamente haciendo $W=\zeta.\zeta^T$ cada elemento de la matriz W nos dará el peso $w_{i,j}$ según la definición anterior:

In [45]:
import numpy as np
import pprint
def signo(x):
    salida=[-1 if bit <0 else 1 for bit in x ]
    return salida
zeta=np.array([[1,1,1,-1,-1,-1]])
W=np.matmul(zeta.T,zeta) #recibe vectores columna la transposición va al revés
print(W)

[[ 1  1  1 -1 -1 -1]
 [ 1  1  1 -1 -1 -1]
 [ 1  1  1 -1 -1 -1]
 [-1 -1 -1  1  1  1]
 [-1 -1 -1  1  1  1]
 [-1 -1 -1  1  1  1]]


Ahora tenemos almacenado un patrón de referencia $\zeta$. Verifiquemos que cuando el mismo es ingresado a la red de Hopfield, es recuperado correctamente:

In [46]:
O=signo(np.dot(W,zeta.T)) #Calculo la salida cuando pongo al patrón de referencia de entrada
print(O)

[1, 1, 1, -1, -1, -1]


Veamos que pasa cuando ingreso a la red de Hopfield con el patrón de referencia, pero con un error en el bit 1:

In [48]:
zeta_error=np.array([[-1,1,1,-1,-1,-1]])
O=signo(np.dot(W,zeta_error.T)) #Calculo la salida cuando pongo al patrón de referencia de entrada con error en el bit 1
print(O)

[1, 1, 1, -1, -1, -1]


Como para calcular cada bit $j$ de salida no solo se pesa la entrada $j$ correspondiente sino que también el resto pesadas por su correlación con la salida $j$, el bit erróneo ha sido corregido. Dejaremos para mas adelante el análisis de qué pasa cuando son mas los bits erróneos que correctos.

## Almacenamiento de varios patrones de referencia  

Veamos cómo almacenar M patrones de referencia $\zeta^\mu$, con $1 \leq \mu \leq M$ .

Propondremos la siguiente ecuación para calcular los pesos $w_{i,j}$, como extensión de la definición dada anteriormente para un solo patrón:

$w_{i,j} = \sum_{\mu=1}^{M} {\zeta^{\mu}_i \zeta^{\mu}_j}$

Donde M es la cantidad de patrones de referencia que queremos almacenar.  

$\zeta^\mu_j$ es bit j del patrón de referencia $\mu$ y puede valer 1 o -1.  
A esta regla de aprendizaje se la conoce como Regla de Hebb.

Esta regla consiste en ir acumulando en el peso $w_{i,j}$ la multiplicación entre el bit $i$ y el bit $j$ de cada uno de los patrones de referencia. Ese peso $w_{i,j}$, nos dará una medida de cómo están correlacionados los bits $i$ y $j$ a lo largo de todos los patrones.

Es interesante que esta regla de aprendizaje permite la incorporación de nuevos patrones a la matriz de pesos a medida que van apareciendo sin necesidad de procesar todo nuevamente, a diferencia del método de Gradient Descent que vimos cuando estudiamos las redes backpropagation. Haciendo uso de esta propiedad, agregaremos el patrón $\zeta^2 = [1,-1,-1,-1,-1,-1]$:

In [49]:
zeta2=np.array([[1,-1,-1,-1,-1,-1]])
W=W+np.matmul(zeta2.T,zeta2) #recibe vectores columna la transposición va al revés
print(W)

[[ 2  0  0 -2 -2 -2]
 [ 0  2  2  0  0  0]
 [ 0  2  2  0  0  0]
 [-2  0  0  2  2  2]
 [-2  0  0  2  2  2]
 [-2  0  0  2  2  2]]


En este caso, M=2. 
Algunas cosas para notar:

- Los elementos de la diagonal de la matriz de pesos **van a valer siempre M** ya que la correlación de un bit $i$ consigo mismo siempre dará 1.   
- La fórmula del cálculo del peso $w_{i,j}$ es siempre igual a la del peso $w_{j,i}$ por lo que en las redes de Hopfield **la matriz de pesos $W$ es siempre simétrica**.

En este caso, la relación entre los bits 1 y 2 del primer patrón de referencia es directa, y en el segundo patrón de referencia es inversa, por lo que $w_{1,2}$ y $w_{2,1}$ tienen valor cero. Esto significa que la entrada $\xi_1$ no me brinda información a la hora de determinar mi salida $O_2$ ni mi entrada $\xi_2$ me briinda información a la hora de determinar mi salida $O_1$. Por otro lado, para ambos patrones, la relación entre los bits 1 y 4 es inversa, por lo que $w_{1,4}$ y $w_{4,1}$ tienen un valor muy negativo.

Veamos si recuperamos correctamente los patrones de referencia utilizados para calcular los pesos:

In [50]:
#Testeamos zeta
O=signo(np.dot(W,zeta.T)) #Calculo la salida cuando pongo al patrón de referencia zeta
print(O)

[1, 1, 1, -1, -1, -1]


In [51]:
#Testeamos zeta 2
O=signo(np.dot(W,zeta2.T)) #Calculo la salida cuando pongo al patrón de referencia zeta
print(O)

[1, -1, -1, -1, -1, -1]


Veamos qué pasa ahora cuando entramos con el patrón de referencia $\zeta$ con un error en el bit 1:

In [52]:
#Testeamos zeta_error
O=signo(np.dot(W,zeta_error.T)) #Calculo la salida cuando pongo al patrón de referencia zeta_error
print(O)

[1, 1, 1, -1, -1, -1]


Por el momento la red está funcionando correctamente. Veamos qué pasa si entramos con el patrón de referencia $\zeta^2$ con un error en el bit 1:

In [53]:
zeta2_error=np.array([[-1,-1,-1,-1,-1,-1]])
O=signo(np.dot(W,zeta2_error.T)) #Calculo la salida cuando pongo al patrón de referencia 2 de entrada con error en el bit 1
print(O)

[1, -1, -1, -1, -1, -1]


El patrón corrompido se corrigió satisfactoriamente.
Veamos ahora qué pasa si entramos con el patrón $\xi=[-1,-1,-1,1,1,-1]$. Siendo que está a distancia de Hamming 3 de $\zeta^2$ y a distancia 5 de $\zeta$, deseamos que la salida de la red para este patrón sea $\zeta^2$. Veamos qué pasa:

In [61]:
xi=np.array([[-1,-1,-1,1,1,-1]])
O=signo(np.dot(W,xi.T)) #Calculo la salida cuando pongo al patrón de referencia 2 de entrada con error en el bit 1
print(O)

[-1, -1, -1, 1, 1, 1]


La salida obtenida no fue ni $\zeta$ ni $\zeta^2$. A estos estados no deseados los denominaremos *estados espurios*, por lo que nuesta memoria asociativa no solo almacena los patrones de referencia que utilizamos para calcular los pesos, sino que parecería tener almacenados otros estados.  
En principio podemos ver que el patrón que nos dió a la salida es igual a $-\zeta$. ¿Cómo es posible que $-\zeta$ se ecuentre almacenado como patrón de referencia? Esto se debe a que la correlación entre los bits de un patrón $\zeta$ y un patrón $-\zeta$ es la misma, ya que todos sus bits se encuentran multiplicados por (-1), lo cual nos da lo mismos valores de pesos $w_{i,j}$ almacenar a $\zeta$ o almacenar a $-\zeta$, ya que:

$$ w_{i,j} = \zeta_i.\zeta_j = -\zeta_i.-\zeta_j $$

Por lo expresado anteriormente, cada vez que almacenamos un patrón de referencia en una red de Hopfield, estamos almacenando su inverso de forma espuria.  
Si bien en principio una red de Hopfield podría ser una herramienta válida para implementar una memoria asociativa, debido a las particularidades que presenta debemos estudiarla mejor y respondernos las siguientes preguntas para poder utilizarla, comprendiendo sus límites:

- ¿Aparecen otros estados espurios además de los patrones de referencia inversos?
- ¿Puedo almacenar todos los patrones que quiero? ¿O la red tiene una capacidad determinada?
- ¿Hay alguna relación entre la cantidad de estados espurios y la cantidad de patrones almacenados?
- ¿Se puede aumentar la capacidad de la red para guardar mas patrones de referencia?
- ¿Se pueden eliminar o minimizar los estados espurios?
- ¿Qué tan robusta es la red? Es decir, ¿Cómo se comporta a medida que empiezo a forzar aleatoriamente algunos pesos $w_{i,j}$ a cero?